In [2]:
import numpy as np 
import pandas as pd
import requests
from tqdm import tqdm
import ast
from time import sleep
from bs4 import BeautifulSoup
data_folder = ''
api = ''

In [3]:
df = pd.read_csv(data_folder+'drug_data.csv')
df.head(1)

,ingredient_id,ingredient_name,product_id,product_name,active_ingredients,company_name
0,/emc/ingredient/1524,"2,4-dichlorobenzyl alcohol",/emc/product/1589/smpc,\n Boots Antiseptic Cream\n...,"allantoin, cetrimide, 2,4-dichlorobenzyl alcohol",THE BOOTS COMPANY PLC


In [4]:
ingredient_df = pd.read_csv(data_folder+'ingredient_data.csv')
print(ingredient_df.shape)
ingredient_df.head()

(2035, 5)


,ingredient_id,ingredient_name,concept_id,concept_code,concept_name
0,/emc/ingredient/1524,"2,4-dichlorobenzyl alcohol",NaN,NaN,NaN
1,/emc/ingredient/3087,5-aminolevulinic acid,NaN,NaN,NaN
2,/emc/ingredient/2642,5-aminolevulinic acid hydrochloride,NaN,NaN,NaN
3,/emc/ingredient/1388,abacavir,1736971.0,190521.0,abacavir
4,/emc/ingredient/1141,abacavir sulfate,19052381.0,221052.0,abacavir sulfate


## Map using Athena RxNorm Data

In [ ]:
#filter
athena = '' #external data folder
athena_map = pd.read_csv(athena+'CONCEPT.csv', delimiter = '\t')
athena_map = athena_map[athena_map.vocabulary_id == 'RxNorm']
athena_ingredients = athena_map[athena_map.concept_class_id.isin(['Ingredient', 'Multiple Ingredients', 'Precise Ingredient'])][['concept_id', 'concept_code', 'concept_name']]
athena_ingredients.head()

<ipython-input-21-0ffa5db50006>:3: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  athena_map = pd.read_csv(athena+'CONCEPT.csv', delimiter = '\t')


,concept_id,concept_code,concept_name
206080,40224159,1000082,alcaftadine
206100,40224763,1000104,incobotulinumtoxinA
206108,46274193,1000112,medroxyprogesterone acetate
206141,46274194,1000146,estradiol cypionate
206192,40225716,1000492,resveratrol


In [ ]:
ingredient_df = ingredient_df.merge(athena_ingredients, left_on = 'ingredient_name', right_on = 'concept_name', how = 'left')
ingredient_df.head()

,ingredient_id,ingredient_name,concept_id,concept_code,concept_name
0,/emc/ingredient/1524,"2,4-dichlorobenzyl alcohol",NaN,NaN,NaN
1,/emc/ingredient/3087,5-aminolevulinic acid,NaN,NaN,NaN
2,/emc/ingredient/2642,5-aminolevulinic acid hydrochloride,NaN,NaN,NaN
3,/emc/ingredient/1388,abacavir,1736971.0,190521,abacavir
4,/emc/ingredient/1141,abacavir sulfate,19052381.0,221052,abacavir sulfate


## UMLS mapping

In [ ]:
#use UMLS mapping
for i, row in tqdm(ingredient_df.iterrows()):
  concept_code = row['concept_code'] 
  if str(concept_code) == 'nan':
    ingredient_name = row['ingredient_name']
    url = 'https://uts-ws.nlm.nih.gov/rest/search/current?string={a}&returnIdType=sourceConcept&apiKey={k}'.format(a=ingredient_name, k=api) 
    result = requests.get(url).json()
    if result['result']['recCount'] > 0:
      for r in result['result']['results']:
        if r['rootSource'] == 'RXNORM':
          ingredient_df.at[i, 'concept_code'] = r['ui']
          ingredient_df.at[i, 'concept_name'] = r['name'] 
          print(concept_code, ingredient_name, r['ui'], r['name'])

70it [00:02, 40.08it/s]

nan aluminium hydroxide 612 aluminum hydroxide


109it [00:03, 30.83it/s]

nan amphotericin b 732 amphotericin B


130it [00:04, 33.66it/s]

...

1996it [01:04, 27.11it/s]

nan vitamin e 11256 vitamin E


2004it [01:04, 30.29it/s]

nan von willebrand factor 11274 von Willebrand factor


2035it [01:06, 30.72it/s]


In [ ]:
ingredient_df.to_csv(data_folder+'ingredient_data.csv', index=False)
ingredient_df.head()

,ingredient_id,ingredient_name,concept_id,concept_code,concept_name
0,/emc/ingredient/1524,"2,4-dichlorobenzyl alcohol",NaN,NaN,NaN
1,/emc/ingredient/3087,5-aminolevulinic acid,NaN,NaN,NaN
2,/emc/ingredient/2642,5-aminolevulinic acid hydrochloride,NaN,NaN,NaN
3,/emc/ingredient/1388,abacavir,1736971.0,190521,abacavir
4,/emc/ingredient/1141,abacavir sulfate,19052381.0,221052,abacavir sulfate


## RxNav ID

In [5]:
ingredient_df = pd.read_csv(data_folder+'ingredient_data.csv')
ingredient_df.head()

,ingredient_id,ingredient_name,concept_id,concept_code,concept_name
0,/emc/ingredient/1524,"2,4-dichlorobenzyl alcohol",NaN,NaN,NaN
1,/emc/ingredient/3087,5-aminolevulinic acid,NaN,NaN,NaN
2,/emc/ingredient/2642,5-aminolevulinic acid hydrochloride,NaN,NaN,NaN
3,/emc/ingredient/1388,abacavir,1736971.0,190521.0,abacavir
4,/emc/ingredient/1141,abacavir sulfate,19052381.0,221052.0,abacavir sulfate


In [23]:
for i, row in tqdm(ingredient_df.iterrows()):
  if str(row['concept_code']) == 'nan':
    name = row['ingredient_name']
    url = 'https://rxnav.nlm.nih.gov/REST/rxcui.json?name={}&search=0'.format(name.replace(' ', '+'))
    r = requests.get(url).json()
    try:
      rxnorm_id = r['idGroup']['rxnormId'][0]
      ingredient_df.at[i, 'concept_code'] = rxnorm_id
    except:
      pass

2035it [04:13,  8.03it/s]


In [25]:
for i, row in tqdm(ingredient_df.iterrows()):
  if str(row['concept_code']) == 'nan':
    name = row['ingredient_name']
    url = 'https://rxnav.nlm.nih.gov/REST/rxcui.json?name={}&search=1'.format(name.replace(' ', '+'))
    r = requests.get(url).json()
    try:
      rxnorm_id = r['idGroup']['rxnormId'][0]
      ingredient_df.at[i, 'concept_code'] = rxnorm_id
    except:
      pass

2035it [02:12, 15.32it/s]


In [45]:
for i, row in tqdm(ingredient_df.iterrows()):
  if str(row['concept_code']) != 'nan' and row['concept_name'] == None:
    code = row['concept_code'] 
    url = 'https://rxnav.nlm.nih.gov/REST/rxcui/{}.json'.format(code)
    r = requests.get(url).json()
    name = r['idGroup']['name']
    ingredient_df.at[i, 'concept_name'] = name

2035it [02:12, 15.30it/s]


In [48]:
ingredient_df.to_csv(data_folder+'ingredient_data.csv', index=False)
ingredient_df.head()

,ingredient_id,ingredient_name,concept_id,concept_code,concept_name
0,/emc/ingredient/1524,"2,4-dichlorobenzyl alcohol",NaN,22906,dichlorobenzyl alcohol
1,/emc/ingredient/3087,5-aminolevulinic acid,NaN,683,aminolevulinic acid
2,/emc/ingredient/2642,5-aminolevulinic acid hydrochloride,NaN,261715,aminolevulinic acid hydrochloride
3,/emc/ingredient/1388,abacavir,1736971.0,190521.0,abacavir
4,/emc/ingredient/1141,abacavir sulfate,19052381.0,221052.0,abacavir sulfate


In [47]:
ingredient_df[ingredient_df.concept_code.isna()]

,ingredient_id,ingredient_name,concept_id,concept_code,concept_name
7,/emc/ingredient/2341,abietis oil,NaN,NaN,NaN
33,/emc/ingredient/3299,adenovirus type 26 encoding the SARS-CoV-2 spi...,NaN,NaN,NaN
41,/emc/ingredient/2586,agnus castus,NaN,NaN,NaN
42,/emc/ingredient/2267,agomelatine,NaN,NaN,NaN
46,/emc/ingredient/2982,Alder pollen (Alnus spp.),NaN,NaN,NaN
...,...,...,...,...,...
2007,/emc/ingredient/2997,Wheat grain,NaN,NaN,NaN
2009,/emc/ingredient/1528,"witch hazel, distilled",NaN,NaN,NaN
2010,/emc/ingredient/2499,"wool alcohols, acetylated",NaN,NaN,NaN
2014,/emc/ingredient/2994,Yorkshire Fog pollen (Holcus lanatus),NaN,NaN,NaN
